### Инициализация lib

In [1]:
import os
import torch

# Base lib
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import snapshot_download

# Quantization
from transformers import (
    BitsAndBytesConfig
    )




# Second base lib
from sentence_transformers import SentenceTransformer



/home/retro0/cyberspace/ENVIRONMENTS/back_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

### Скачивание модели 

In [3]:
local_dir = "../modelsPack/Qwen3" #  "../modelsPack/DeepSeek"
repo_id = "Qwen/Qwen3-14B" # "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

local_model_path = snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
)

Returning existing local_dir `../modelsPack/Qwen3` as remote repo cannot be accessed in `snapshot_download` ((MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/models/Qwen/Qwen3-14B/revision/main (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78f8cd726ab0>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 338d27ca-2606-45c8-9246-cc8487c7a6e2)')).


### Конфиг Квантизации

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True
)

### Модель с квантизацией

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    quantization_config = quantization_config,
    torch_dtype ="auto",
    device_map = DEVICE,
)

tokenizer = AutoTokenizer.from_pretrained(
    local_model_path
)

Loading checkpoint shards: 100%|██████████| 8/8 [00:55<00:00,  6.94s/it]


In [9]:
model_inputs = tokenizer(
    ["Придумай небольшой рассказ на 10 взаимосвязанных предложений, про молодого инженера программиста."],
    return_tensors="pt"
).to(DEVICE)


generate_ids = model.generate(
    **model_inputs, 
    max_new_tokens=4096,
    num_beams = 1,
    #num_beam_groups = 3,
    #diversity_penalty = 1.0,
    do_sample = True,
    use_cache=True
    )
print(f"{tokenizer.batch_decode(generate_ids)[0]}")

Придумай небольшой рассказ на 10 взаимосвязанных предложений, про молодого инженера программиста. Он живет в однокомнатной квартире, где в качестве холодильника использует старую, но надёжную модель 1990-х годов, и в качестве холодильника использует старую, но надёжную модель 1990-х годов. Он работает на дому, и когда его ноутбук перегревается, он открывает дверцу холодильника и выкладывает туда ноутбук, чтобы он остыл. Он думает, что это необычное решение, но оно работает. Один день он замечает, что в холодильнике осталось немного йогурта, и решает его съесть. Но тут он замечает, что ноутбук, который он оставил в холодильнике, теперь работает с непривычно высокой скоростью. Он понимает, что холодильник охлаждает не только ноутбук, но и его мозг, и решает использовать эту находку. Он начинает охлаждать мозг, чтобы быть более продуктивным, но с каждым днём всё больше и больше. Он понимает, что он уже не может думать, когда его мозг нагрет, и что теперь он должен охлаждать мозг, как ноут

In [9]:
torch.cuda.empty_cache()